In [1]:
# Importing all needed libraries.
from string import ascii_uppercase
from operator import and_
from functools import reduce

In [7]:
# Creatting the class Grammar.
class Grammar:
    def __init__(self, Vn, Vt, P, S):
        ''' Setting up the Grammar '''
        self.Vn = Vn
        self.Vt = Vt
        self.P = P
        self.S = S
        self.free_uppercase = [letter for letter in ascii_uppercase if letter not in self.Vn]
    
    def __add_S0(self):
        ''' This function adds a starting state '''
        add_S0 = False
        for i in range(len(self.P)):
            if self.P[i][0] == self.S:
                add_S0 = True
        if add_S0:
            self.P.append(('S0', self.S))
            self.S = 'S0'
    
    def __eliminate_terminals(self):
        ''' This funtion eliminates the terminals '''
        to_eliminate = set()
        for production in self.P:
            to_eliminate.update(set(production[1]).intersection(set(self.Vt)))
        to_eliminate = list(to_eliminate)
        for production in self.P:
            if production[1] in self.Vt and production[1] in to_eliminate:
                to_eliminate.remove(production[1])
        for element in to_eliminate:
            self.P.append((self.free_uppercase[0], element))
            self.free_uppercase.pop(0)
        back_mapping = dict()
        for i in range(len(self.P)):
            if self.P[i][1] in self.Vt:
                back_mapping[self.P[i][1]] = self.P[i][0]
        for i in range(len(self.P)):
            for key in back_mapping:
                if key in self.P[i][1] and len(self.P[i][1]) > 1:
                    self.P[i][1] = self.P[i][1].replace(key, back_mapping[key])
    
    def __eliminate_more_then_2(self):
        ''' This function eliminates elements with more than 2 symbols '''
        while True:
            with_2_symbols = []
            with_more_2_symbols = []
            for i in range(len(self.P)):
                if len(self.P[i][1]) == 2:
                    with_2_symbols.append(i)
                elif len(self.P[i][1]) > 2:
                    with_more_2_symbols.append(i)
            if len(with_more_2_symbols) == 0:
                break
            for i in with_2_symbols:
                for j in with_more_2_symbols:
                    if self.P[i][1] in self.P[j][1] and i != j:
                        self.P[j][1] = self.P[j][1].replace(self.P[i][1], self.P[i][0])
            if len(with_more_2_symbols) != 0:
                substrings = []
                for i in with_more_2_symbols:
                    substrings.append([])
                    for j in range(len(self.P[i][1]) -1):
                        substrings[-1].append(self.P[i][1][j:j+2])
                substrings = [set(sub) for sub in substrings]
                common_strings = list(reduce(and_, substrings))
                for common_string in common_strings:
                    self.P.append([self.free_uppercase[0], common_string])
                    self.free_uppercase.pop(0)
        return self.P
    
    def to_chomsky(self):
        ''' Convertion to chomsky form '''
        self.__add_S0()
        self.P = [pair for pair in self.P if pair[1] != 'epsilon']
        self.__eliminate_terminals()
        self.__eliminate_more_then_2()
        return self.P

In [8]:
dictionary = [['S', 'AC'], ['S', 'bA'], ['S', 'B'], ['S', 'aA'], ['A', 'epsilon'],
              ['A', 'aS'], ['A', 'ABab'], ['B', 'a'], ['B', 'bS'], ['C', 'abC'], 
              ['D', 'AB']]

In [9]:
Vt = ['a', 'b']
Vn = ['S', 'A', 'B', 'C', 'D']

In [10]:
gr = Grammar(Vn, Vt, dictionary, 'S')

In [11]:
gr.to_chomsky()

[['S', 'AC'],
 ['S', 'EA'],
 ['S', 'B'],
 ['S', 'BA'],
 ['A', 'BS'],
 ['A', 'DF'],
 ['B', 'a'],
 ['B', 'ES'],
 ['C', 'FC'],
 ['D', 'AB'],
 ('S0', 'S'),
 ('E', 'b'),
 ['F', 'BE']]